<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-the-structure-and-settings" data-toc-modified-id="Import-the-structure-and-settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import the structure and settings</a></span><ul class="toc-item"><li><span><a href="#Show-the-slices" data-toc-modified-id="Show-the-slices-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Show the slices</a></span></li></ul></li><li><span><a href="#Define-the-model-and-solve-for-dwells" data-toc-modified-id="Define-the-model-and-solve-for-dwells-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define the model and solve for dwells</a></span></li><li><span><a href="#Save-the-stream" data-toc-modified-id="Save-the-stream-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save the stream</a></span><ul class="toc-item"><li><span><a href="#Load-the-stream" data-toc-modified-id="Load-the-stream-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load the stream</a></span></li></ul></li></ul></div>

In [12]:
%matplotlib qt
import f3ast
from f3ast import deep_reload
deep_reload(f3ast)
from f3ast import *
from time import time

### Import the structure and settings

In [22]:
# file_path = 'testing/FunktyBall.stl'
# file_path = 'testing/temp_testing_struct_1px.stl'
file_path = 'testing/hand_ok_built.stl'

settings = load_settings()
struct = Structure.from_file(file_path, **settings["structure"])
# struct.rescale(5)
struct.show()
# struct.plot_mpl()

#### Show the slices

In [23]:

# deep_reload(f3ast)
# from f3ast import *
struct = Structure.from_file(file_path, **settings["structure"])

# struct.rescale(5)
t0 = time()
struct.generate_slices()
print(time()-t0)

Slicing...
Sliced
44.42999839782715


In [14]:
struct.plot_slices()

(<Axes3D:>, <mpl_toolkits.mplot3d.art3d.Path3DCollection at 0x25aa84ddf70>)

### Define the model and solve for dwells

In [24]:

# deep_reload(f3ast)
# from f3ast import *

# settings = load_settings()
gr = 0.15
k = 1
sigma = 4.4
model = DDModel(struct, gr, k, sigma, **settings['dd_model'])

t0 = time()
stream_builder, dwell_solver = StreamBuilder.from_model(model, **settings['stream_builder'])
print(time()-t0)
dwell_solver.print_total_time()

Solving for dwells...
Solved
329.7645857334137
Total stream time:  2:14:41.135163


In [26]:
2*3600+14*60+41

8081

In [27]:
np.max(np.concatenate(model.resistance))

0.26028318704538106

In [20]:
ax, sc = dwell_solver.show_solution()

In [21]:
resistance = model.resistance
ax, sc = struct.plot_slices(c=np.concatenate(resistance), cmap='hot')
plt.colorbar(sc, shrink=0.8)

In [7]:
plt.figure()
res = [np.mean(r) for r in resistance]
plt.plot(res)

### Save the stream

In [8]:
save_path = 'test1'

strm = stream_builder.get_stream()
strm.write(save_path)
save_build(save_path, dwell_solver, stream_builder)

#### Load the stream

In [11]:
load_path = 'test1.pickle'
strm.from_file('test1.str')
dwell_solver, stream_builder = load_build(load_path)